In [1]:
import ants
from petpal.kinetic_modeling.parametric_images import apply_rtm2_to_all_voxels
from petpal.utils.time_activity_curve import TimeActivityCurveFromFile
from petpal.preproc.image_operations_4d import brain_mask

In [2]:
wmref = TimeActivityCurveFromFile(tac_path='/export/scratch1/PETPAL/03Dec24/sub-PIB07018_ses-VYr12/tacs/WMRef_tac.tsv')

In [3]:
wmref.tac_times_in_minutes

array([  0.45776667,   0.5411    ,   0.62443333,   0.70776667,
         0.7911    ,   0.87443333,   0.95776667,   1.0411    ,
         1.12443333,   1.20776667,   1.2911    ,   1.37443333,
         1.45776667,   1.5411    ,   1.62443333,   1.70776667,
         1.7911    ,   1.87443333,   1.95776667,   2.0411    ,
         2.12443333,   2.20776667,   2.2911    ,   2.37443333,
         2.45776667,   2.5411    ,   2.62443333,   2.70776667,
         2.7911    ,   2.87443333,   2.95776667,   3.0411    ,
         3.12443333,   3.20776667,   3.2911    ,   3.37443333,
         3.45776667,   3.62443333,   3.7911    ,   3.9586    ,
         4.12526667,   4.29193333,   4.4586    ,   4.62526667,
         4.79193333,   4.9586    ,   5.12526667,   5.29193333,
         5.4586    ,  10.4586    ,  15.4586    ,  20.4586    ,
        25.4586    ,  30.4586    ,  35.4586    ,  40.4586    ,
        45.4586    ,  50.4586    ,  55.45888333,  60.45888333,
        65.45888333,  70.45888333,  75.45888333,  80.45

In [4]:
pet_file = '/export/scratch1/PETPAL/03Dec24/sub-PIB07018_ses-VYr12/sub-PIB07018_ses-VYr12_moco.nii.gz'
pet_image = ants.image_read(filename=pet_file)
pet_array = pet_image.numpy()
mask_file = '/tmp/pet_mask.nii.gz'
mask_image = ants.image_read(mask_file)
mask_array = mask_image.numpy()

brain_mask(
    input_image_4d_path=pet_file,
    out_image_path=mask_file,
    atlas_image_path='/data/petsun43/data1/atlas/MNI152/MNI152_T1_2mm.nii',
    atlas_mask_path='/data/petsun43/data1/atlas/MNI152/MNI152_T1_2mm_brain_mask.nii.gz'
)

In [ ]:
srtm2_array = apply_rtm2_to_all_voxels(
    tac_times_in_minutes=wmref.tac_times_in_minutes,
    tgt_image=pet_array,
    ref_tac_vals=wmref.tac_vals,
    mask_img=mask_array,
    method='srtm2'
)

In [8]:
srtm2_image = ants.from_numpy(
    data=srtm2_array,
    origin=pet_image.origin,
    spacing=pet_image.spacing,
    direction=pet_image.direction
)

In [ ]:
srtm2_image

In [10]:
ants.image_write(image=srtm2_image,filename='/tmp/srtm2_example.nii.gz')

In [5]:
frtm2_array = apply_rtm2_to_all_voxels(
    tac_times_in_minutes=wmref.tac_times_in_minutes,
    tgt_image=pet_array,
    ref_tac_vals=wmref.tac_vals,
    mask_img=mask_array,
    method='frtm2',
    r1_bounds=[0.5,0,10],
    k3_bounds=[0.5,0,1],
    k4_bounds=[0.5,0,1],
    k2_prime=0.017
)

In [6]:
frtm2_image = ants.from_numpy(
    data=frtm2_array,
    origin=pet_image.origin,
    spacing=pet_image.spacing,
    direction=pet_image.direction
)
ants.image_write(image=frtm2_image,filename='/tmp/frtm2_example.nii.gz')